In [1]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint

import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from scipy.io import wavfile
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [2]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [3]:
split_txt_val_path = r'.\preprocessed_data\LibriTTS\val_ming.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [4]:
print(len(val_uids))

512


In [5]:
val_duration_folder = './preprocessed_data/LibriTTS/duration/val'
val_energy_folder = './preprocessed_data/LibriTTS/energy/val'
val_mel_folder = './preprocessed_data/LibriTTS/mel/val'
val_pitch_folder = './preprocessed_data/LibriTTS/pitch/val'

os.makedirs(val_duration_folder, exist_ok=True)
os.makedirs(val_energy_folder, exist_ok=True)
os.makedirs(val_mel_folder, exist_ok=True)
os.makedirs(val_pitch_folder, exist_ok=True)


In [6]:
for uid in tqdm(val_uids):
    speaker_id = uid.split('_')[0]
    src = rf".\preprocessed_data\LibriTTS\duration\{speaker_id}-duration-{uid}.npy"
    dst = f"{val_duration_folder}"
    shutil.copy(src, dst)

for uid in tqdm(val_uids):
    speaker_id = uid.split('_')[0]
    src = rf".\preprocessed_data\LibriTTS\energy\{speaker_id}-energy-{uid}.npy"
    dst = f"{val_energy_folder}"
    shutil.copy(src, dst)

for uid in tqdm(val_uids):
    speaker_id = uid.split('_')[0]
    src = rf".\preprocessed_data\LibriTTS\mel\{speaker_id}-mel-{uid}.npy"
    dst = f"{val_mel_folder}"
    shutil.copy(src, dst)

for uid in tqdm(val_uids):
    speaker_id = uid.split('_')[0]
    src = rf".\preprocessed_data\LibriTTS\pitch\{speaker_id}-pitch-{uid}.npy"
    dst = f"{val_pitch_folder}"
    shutil.copy(src, dst)

  0%|          | 0/512 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

  0%|          | 0/512 [00:00<?, ?it/s]

In [3]:
from utils.model import get_model, get_vocoder, get_param_num, vocoder_infer
from utils.tools import to_device, log, synth_one_sample, expand, plot_mel
from model import FastSpeech2Loss
from dataset import Dataset
# from utils.auto_tqdm import tqdm
from evaluate import evaluate

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/LibriTTS/0714lb_preprocess.yaml -m ./config/LibriTTS/0714lb_model.yaml -t ./config/LibriTTS/0714lb_train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())

In [10]:
pprint(args)
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)
print("Prepare training ...")

preprocess_config, model_config, train_config = configs

Namespace(restore_step=0, preprocess_config='./config/LibriTTS/0714lb_preprocess.yaml', model_config='./config/LibriTTS/0714lb_model.yaml', train_config='./config/LibriTTS/0714lb_train.yaml')
Prepare training ...


In [11]:
ckpt_path = r'./output/LibriTTS/LibriTTS_800000.pth.tar'
ckpt = torch.load(ckpt_path)

In [12]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)
model.load_state_dict(ckpt["model"], strict=False)
model.to(device)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)

=> Not using speaker embeddings.
False
None
Number of FastSpeech2 Parameters: 35391169


In [13]:
# Load vocoder
vocoder = get_vocoder(model_config, device)
step = args.restore_step + 1
model.eval()
print()
dataset = Dataset(
    "val_ming.txt", 'val', preprocess_config, train_config, sort=False, drop_last=False
)

Removing weight norm...



In [14]:
len(dataset)

512

In [15]:
batch_size = train_config["optimizer"]["batch_size"]
batch_size = 1

In [16]:
loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=dataset.collate_fn,
    )
# Get loss function
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)

In [17]:
os.path.join(train_config['path']['result_path'], 'plots')

'./output/0714lb/result/LibriTTS\\plots'

In [18]:
output_plot_path = os.path.join(train_config['path']['result_path'], 'plot')
output_mel_syn_path = os.path.join(train_config['path']['result_path'], 'mel', 'syn')
output_mel_gt_path = os.path.join(train_config['path']['result_path'], 'mel', 'gt')
output_wav_syn_path = os.path.join(train_config['path']['result_path'], 'wav', 'synthesized')
output_wav_rec_path = os.path.join(train_config['path']['result_path'], 'wav', 'reconstructed')
os.makedirs(output_plot_path, exist_ok=True)
os.makedirs(output_mel_syn_path, exist_ok=True)
os.makedirs(output_mel_gt_path, exist_ok=True)
os.makedirs(output_wav_syn_path, exist_ok=True)
os.makedirs(output_wav_rec_path, exist_ok=True)

In [19]:
for batchs in tqdm(loader):
    for targets in batchs:
        targets = to_device(targets, device)
        with torch.no_grad():
            predictions = model(*(targets[2:]))
        basenames = targets[0]
        for i in range(len(predictions[0])):
            basename = basenames[i]
            src_len = predictions[8][i].item()
            mel_len = predictions[9][i].item()
            mel_prediction = predictions[1][i, :mel_len].detach().transpose(0, 1)
            mel_target = targets[6][i, :mel_len].detach().transpose(0, 1)

            torch.save(mel_prediction.cpu(), os.path.join(output_mel_syn_path, f"{basename}.pt"))
            torch.save(mel_target.cpu(), os.path.join(output_mel_gt_path, f"{basename}.pt"))
            
            
            duration = predictions[5][i, :src_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["pitch"]["feature"] == "phoneme_level":
                pitch = predictions[2][i, :src_len].detach().cpu().numpy()
                pitch = expand(pitch, duration)
            else:
                pitch = predictions[2][i, :mel_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["energy"]["feature"] == "phoneme_level":
                energy = predictions[3][i, :src_len].detach().cpu().numpy()
                energy = expand(energy, duration)
            else:
                energy = predictions[3][i, :mel_len].detach().cpu().numpy()

            with open(os.path.join(preprocess_config["path"]["preprocessed_path"], 
                                   "stats.json")) as f:
                stats = json.load(f)
                stats = stats["pitch"] + stats["energy"][:2]
                                       
            fig = plot_mel(
                [
                    (mel_prediction.cpu().numpy(), pitch, energy),
                    (mel_target.cpu().numpy(), pitch, energy),
                ],
                stats,
                ["Synthetized Spectrogram", "Ground-Truth Spectrogram"],
            )
            ### TODO: change to svg
            plt.savefig(os.path.join(output_plot_path, f"{basename}.png"))
            plt.close()

        # from .model import vocoder_infer

        mel_predictions = predictions[1].transpose(1, 2)
        mel_targets = targets[6].transpose(1, 2)
        
        lengths = predictions[9] * preprocess_config["preprocessing"]["stft"]["hop_length"]
        wav_predictions = vocoder_infer(
            mel_predictions, vocoder, model_config, preprocess_config, lengths=lengths
        )
        wav_targets = vocoder_infer(
        mel_targets, vocoder, model_config, preprocess_config, lengths=lengths
    )
    
        sampling_rate = preprocess_config["preprocessing"]["audio"]["sampling_rate"]
        for wav, basename in zip(wav_predictions, basenames):
            wavfile.write(os.path.join(output_wav_syn_path, f"{basename}.wav"), sampling_rate, wav)
        for wav, basename in zip(wav_targets, basenames):
            wavfile.write(os.path.join(output_wav_rec_path, f"{basename}.wav"), sampling_rate, wav)

        
    #     break
    # break

  0%|          | 0/512 [00:00<?, ?it/s]

# MCD

In [5]:
import os, sys, glob, shutil
import os.path as op
import argparse
import json
import yaml
import numpy as np
import pandas as pd
from pprint import pprint
import os.path as op

import torch
import torch.nn.functional as F
import torchaudio
import argparse
import fnmatch
import logging
import multiprocessing as mp
from typing import Dict, List, Tuple

import librosa
import pysptk
import pyworld as pw
import soundfile as sf
from fastdtw import fastdtw
from scipy import spatial

In [6]:
split_txt_val_path = r'.\preprocessed_data\LibriTTS\val_ming.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [7]:
len(val_uids)

512

In [8]:
val_uids[0]

'5400_3587_000037_000001'

In [9]:
recon_folder = r'.\output\0714lb\result\LibriTTS\wav\reconstructed'
syn_folder = r'.\output\0714lb\result\LibriTTS\wav\synthesized'
gt_folder = r'.\Data\LibriTTS_val'

In [10]:
# sr = 22050
# melkwargs = {
#     "n_fft": int(0.05 * sr), "win_length": int(0.05 * sr),
#     "hop_length": int(0.0125 * sr), "f_min": 20,
#     "n_mels": 80, "window_fn": torch.hann_window
# }
# mfcc_fn = torchaudio.transforms.MFCC(
#     sr, n_mfcc=13, log_mels=True, melkwargs=melkwargs
# )

In [11]:
# %%time
# uids = []
# MCDs_recon = []
# for uid in tqdm(val_uids):
#     uids.append(uid)
#     wav_1, sr = torchaudio.load(op.join(recon_folder, f"{uid}.wav"))
#     wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
#     wav_1 = wav_1.squeeze()
#     wav_2 = wav_2.squeeze()
#     mel_1 = mfcc_fn(wav_1).T.numpy()
#     mel_2 = mfcc_fn(wav_2).T.numpy()
#     # DTW
#     _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
#     twf = np.array(path).T
#     mel_2 = mel_2[twf[0]]
#     mel_1 = mel_1[twf[1]]
#     # We sum the squared differences over the first K MFCCs, skipping ct,0
#     mel_1 = mel_1[:, 1:]
#     mel_2 = mel_2[:, 1:]
#     result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
#     MCDs_recon.append(result)

In [12]:
%%time

def sptk_extract(
    x: np.ndarray,
    fs: int,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 25,
    mcep_alpha: float = 0.41,
    is_padding: bool = False,
) -> np.ndarray:
    """Extract SPTK-based mel-cepstrum.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Sampling rate
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).
        is_padding (bool): Whether to pad the end of signal (default=False).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).

    """
    # perform padding
    if is_padding:
        n_pad = n_fft - (len(x) - n_fft) % n_shift
        x = np.pad(x, (0, n_pad), "reflect")

    # get number of frames
    n_frame = (len(x) - n_fft) // n_shift + 1

    # get window function
    win = pysptk.sptk.hamming(n_fft)

    # check mcep and alpha
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)

    # calculate spectrogram
    mcep = [
        pysptk.mcep(
            x[n_shift * i : n_shift * i + n_fft] * win,
            mcep_dim,
            mcep_alpha,
            eps=1e-6,
            etype=1,
        )
        for i in range(n_frame)
    ]

    return np.stack(mcep)


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")





uids = []
MCDs_recon = []
for uid in tqdm(val_uids):
    uids.append(uid)
    gen_x, gen_fs = sf.read(op.join(recon_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(gt_folder, f"{uid}.wav"), dtype="int16")
    # print(gen_fs, gt_fs)
    fs = gen_fs
    if gen_fs != gt_fs:
        gt_x = librosa.resample(gt_x.astype(float), orig_sr=gt_fs, target_sr=gen_fs)

    # extract ground truth and converted features
    gen_mcep = sptk_extract(
        x=gen_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep = sptk_extract(
        x=gt_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_mcep_dtw = gen_mcep[twf[0]]
    gt_mcep_dtw = gt_mcep[twf[1]]

    # MCD
    diff2sum = np.sum((gen_mcep_dtw - gt_mcep_dtw) ** 2, 1)
    mcd = np.mean(10.0 / np.log(10.0) * np.sqrt(2 * diff2sum), 0)
    result = mcd
    MCDs_recon.append(result)

  0%|          | 0/512 [00:00<?, ?it/s]

CPU times: total: 5min 39s
Wall time: 7min 16s


In [13]:
# %%time
# uids = []
# MCDs_gt = []
# for uid in tqdm(val_uids):
#     uids.append(uid)
#     wav_1, sr = torchaudio.load(op.join(gt_folder, f"{uid}.wav"))
#     wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
#     wav_1 = wav_1.squeeze()
#     wav_2 = wav_2.squeeze()
#     mel_1 = mfcc_fn(wav_1).T.numpy()
#     mel_2 = mfcc_fn(wav_2).T.numpy()
#     # DTW
#     _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
#     twf = np.array(path).T
#     mel_2 = mel_2[twf[0]]
#     mel_1 = mel_1[twf[1]]
#     # We sum the squared differences over the first K MFCCs, skipping ct,0
#     mel_1 = mel_1[:, 1:]
#     mel_2 = mel_2[:, 1:]
#     result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
#     MCDs_gt.append(result)

In [14]:
%%time

def sptk_extract(
    x: np.ndarray,
    fs: int,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 25,
    mcep_alpha: float = 0.41,
    is_padding: bool = False,
) -> np.ndarray:
    """Extract SPTK-based mel-cepstrum.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Sampling rate
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).
        is_padding (bool): Whether to pad the end of signal (default=False).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).

    """
    # perform padding
    if is_padding:
        n_pad = n_fft - (len(x) - n_fft) % n_shift
        x = np.pad(x, (0, n_pad), "reflect")

    # get number of frames
    n_frame = (len(x) - n_fft) // n_shift + 1

    # get window function
    win = pysptk.sptk.hamming(n_fft)

    # check mcep and alpha
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)

    # calculate spectrogram
    mcep = [
        pysptk.mcep(
            x[n_shift * i : n_shift * i + n_fft] * win,
            mcep_dim,
            mcep_alpha,
            eps=1e-6,
            etype=1,
        )
        for i in range(n_frame)
    ]

    return np.stack(mcep)


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")





uids = []
MCDs_gt = []
for uid in tqdm(val_uids):
    uids.append(uid)
    gen_x, gen_fs = sf.read(op.join(syn_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(gt_folder, f"{uid}.wav"), dtype="int16")
    # print(gen_fs, gt_fs)
    fs = gen_fs
    if gen_fs != gt_fs:
        gt_x = librosa.resample(gt_x.astype(float), orig_sr=gt_fs, target_sr=gen_fs)

    # extract ground truth and converted features
    gen_mcep = sptk_extract(
        x=gen_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep = sptk_extract(
        x=gt_x,
        fs=fs,
        n_fft=512,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_mcep_dtw = gen_mcep[twf[0]]
    gt_mcep_dtw = gt_mcep[twf[1]]

    # MCD
    diff2sum = np.sum((gen_mcep_dtw - gt_mcep_dtw) ** 2, 1)
    mcd = np.mean(10.0 / np.log(10.0) * np.sqrt(2 * diff2sum), 0)
    result = mcd
    MCDs_gt.append(result)

  0%|          | 0/512 [00:00<?, ?it/s]

CPU times: total: 5min 24s
Wall time: 7min 20s


In [15]:
lb_mcd_df = pd.DataFrame({
    'uid': uids,
    'MCD_recon': MCDs_recon,
    'MCD_gt': MCDs_gt,
})

In [16]:
lb_mcd_df.to_csv(r".\jupyter_walkthrough\metrics\MCD_0714lb_800k_new.csv")

In [17]:
print(f"MCDs_recon mean on 0714lb_800k: {torch.tensor(MCDs_recon).mean()}")
print(f"MCDs_recon std on 0714lb_800k: {torch.tensor(MCDs_recon).std()}")

print(f"MCDs_gt mean on 0714lb_800k: {torch.tensor(MCDs_gt).mean()}")
print(f"MCDs_gt std on 0714lb_800k: {torch.tensor(MCDs_gt).std()}")

MCDs_recon mean on 0714lb_800k: 5.648897674732002
MCDs_recon std on 0714lb_800k: 1.6735133209184978
MCDs_gt mean on 0714lb_800k: 8.619400217271503
MCDs_gt std on 0714lb_800k: 1.7607610658468404


In [18]:
lb_mcd_df.nsmallest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
394,7117_86732_000004_000002,3.079804,6.036933
262,2618_138042_000045_000002,3.121507,6.718762
69,2512_157242_000069_000000,3.131460,7.538399
506,1445_139404_000007_000004,3.156019,6.967686
101,4054_11077_000004_000004,3.166626,7.140275


In [19]:
lb_mcd_df.nlargest(5, 'MCD_recon')

,uid,MCD_recon,MCD_gt
178,369_125883_000004_000001,15.447646,17.432105
421,2004_147173_000029_000003,13.154249,11.902160
74,209_157830_000013_000003,12.544040,11.344743
7,954_130627_000038_000001,12.527503,14.996965
125,369_125883_000053_000000,12.090939,17.993798


# log f0

In [13]:
# https://github.com/espnet/espnet/blob/3e0dad524d62ccd45e067e9b36049f2214ea972a/egs2/TEMPLATE/asr1/pyscripts/utils/evaluate_f0.py

In [21]:
def world_extract(
    x: np.ndarray,
    fs: int,
    f0min: int = 40,
    f0max: int = 800,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 25,
    mcep_alpha: float = 0.41,
) -> np.ndarray:
    """Extract World-based acoustic features.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Minimum f0 value (default=40).
        f0 (int): Maximum f0 value (default=800).
        n_shift (int): Shift length in point (default=256).
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).
        ndarray: F0 sequence (N,).

    """
    # extract features
    x = x.astype(np.float64)
    f0, time_axis = pw.harvest(
        x,
        fs,
        f0_floor=f0min,
        f0_ceil=f0max,
        frame_period=n_shift / fs * 1000,
    )
    sp = pw.cheaptrick(x, f0, time_axis, fs, fft_size=n_fft)
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)
    mcep = pysptk.sp2mc(sp, mcep_dim, mcep_alpha)

    return mcep, f0


def _get_basename(path: str) -> str:
    return os.path.splitext(os.path.split(path)[-1])[0]


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")

In [22]:
uids = []
logf0_rmses_recon = []
logf0_corrs_recon = []

for uid in tqdm(val_uids):
    uids.append(uid)
    # load wav file as int16
    gen_x, gen_fs = sf.read(op.join(syn_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(recon_folder, f"{uid}.wav"), dtype="int16")
    fs = gen_fs
    if gen_fs != gt_fs:
        gt_x = librosa.resample(gt_x.astype(float), orig_sr=gt_fs, target_sr=gen_fs)
    # extract ground truth and converted features
    gen_mcep, gen_f0 = world_extract(
        x=gen_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep, gt_f0 = world_extract(
        x=gt_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_f0_dtw = gen_f0[twf[0]]
    gt_f0_dtw = gt_f0[twf[1]]
    
    # Get voiced part
    nonzero_idxs = np.where((gen_f0_dtw != 0) & (gt_f0_dtw != 0))[0]
    gen_f0_dtw_voiced = np.log(gen_f0_dtw[nonzero_idxs])
    gt_f0_dtw_voiced = np.log(gt_f0_dtw[nonzero_idxs])

    # log F0 RMSE
    log_f0_rmse = np.sqrt(np.mean((gen_f0_dtw_voiced - gt_f0_dtw_voiced) ** 2))
    # print(f"{uid} {log_f0_rmse:.4f}")

    # log F0 corr
    log_f0_corr = np.corrcoef(gen_f0_dtw_voiced, gt_f0_dtw_voiced)[0][1]
    # print(f"{uid} {log_f0_corr:.4f}")

    logf0_rmses_recon.append(log_f0_rmse)
    logf0_corrs_recon.append(log_f0_corr)



  0%|          | 0/512 [00:00<?, ?it/s]

D:\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
D:\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
D:\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
D:\Programs\Python\Python311

In [25]:
uids = []
logf0_rmses_gt = []
logf0_corrs_gt = []

for uid in tqdm(val_uids):
    uids.append(uid)
    # load wav file as int16
    gen_x, gen_fs = sf.read(op.join(syn_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(gt_folder, f"{uid}.wav"), dtype="int16")
    fs = gen_fs
    if gen_fs != gt_fs:
        gt_x = librosa.resample(gt_x.astype(float), orig_sr=gt_fs, target_sr=gen_fs)
    # extract ground truth and converted features
    gen_mcep, gen_f0 = world_extract(
        x=gen_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep, gt_f0 = world_extract(
        x=gt_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_f0_dtw = gen_f0[twf[0]]
    gt_f0_dtw = gt_f0[twf[1]]
    
    # Get voiced part
    nonzero_idxs = np.where((gen_f0_dtw != 0) & (gt_f0_dtw != 0))[0]
    gen_f0_dtw_voiced = np.log(gen_f0_dtw[nonzero_idxs])
    gt_f0_dtw_voiced = np.log(gt_f0_dtw[nonzero_idxs])

    # log F0 RMSE
    log_f0_rmse = np.sqrt(np.mean((gen_f0_dtw_voiced - gt_f0_dtw_voiced) ** 2))
    # print(f"{uid} {log_f0_rmse:.4f}")

    # log F0 corr
    log_f0_corr = np.corrcoef(gen_f0_dtw_voiced, gt_f0_dtw_voiced)[0][1]
    # print(f"{uid} {log_f0_corr:.4f}")

    logf0_rmses_gt.append(log_f0_rmse)
    logf0_corrs_gt.append(log_f0_corr)

  0%|          | 0/512 [00:00<?, ?it/s]

In [27]:
lb_logf0_df = pd.DataFrame({
    'uid': uids,
    'logf0_rmse_recon': logf0_rmses_recon,
    'logf0_corr_recon': logf0_corrs_recon,
    'logf0_rmse_gt': logf0_rmses_gt,
    'logf0_corr_gt': logf0_corrs_gt,
})
lb_logf0_df.to_csv(r".\jupyter_walkthrough\metrics\logF0_LB_new.csv")

In [29]:
lb_logf0_df.shape

(512, 5)

In [31]:
lb_logf0_df = lb_logf0_df.dropna()

In [34]:
print(f"logf0_rmse_recon mean on LibriTTS: {lb_logf0_df['logf0_rmse_recon'].values.mean()}")
print(f"logf0_rmse_recon std on LibriTTS: {lb_logf0_df['logf0_rmse_recon'].values.std()}")
print(f"logf0_corr_recon mean on LibriTTS: {lb_logf0_df['logf0_corr_recon'].values.mean()}")
print(f"logf0_corr_recon std on LibriTTS: {lb_logf0_df['logf0_corr_recon'].values.std()}")

print(f"logf0_rmse_gt mean on LibriTTS: {lb_logf0_df['logf0_rmse_gt'].values.mean()}")
print(f"logf0_rmse_gt std on LibriTTS: {lb_logf0_df['logf0_rmse_gt'].values.std()}")
print(f"logf0_corr_gt mean on LibriTTS: {lb_logf0_df['logf0_corr_gt'].values.mean()}")
print(f"logf0_corr_gt std on LibriTTS: {lb_logf0_df['logf0_corr_gt'].values.std()}")

logf0_rmse_recon mean on LibriTTS: 0.2670626798174982
logf0_rmse_recon std on LibriTTS: 0.11483676962626277
logf0_corr_recon mean on LibriTTS: 0.5683756745963985
logf0_corr_recon std on LibriTTS: 0.24157468871441268
logf0_rmse_gt mean on LibriTTS: 0.27041593332575575
logf0_rmse_gt std on LibriTTS: 0.12007872804610982
logf0_corr_gt mean on LibriTTS: 0.5523046469028201
logf0_corr_gt std on LibriTTS: 0.23692520122464472
